In [3128]:
wordList = [
    [2],                                                 #i
    [17, 7, 12, 23],                                     #th?k
    [11, 14, 15],                                        #you
    [8, 14, 4],                                          #for
    [23, 16, 16, 1, 2, 12, 6],                           #keeping 
    [3, 16],                                             #me
    [24, 25, 8, 16],                                     #safe
    [21, 7, 2, 0, 16, 11, 14, 15],                       #?hile?o?
    [8, 14, 15, 12, 9],                                  #fo???
    [21, 7, 25, 17],                                     #?hat
    [11, 14, 15],                                        #?o?
    [12, 16, 16, 9, 16, 9],                              #?ee?e?
    [17, 14, 24, 14, 0, 28, 16],                         #tosolve
    [17, 7, 2, 24],                                      #this
    [17, 7, 16],                                         #the
    [27, 14, 9, 16],                                     #code
    [17, 14],                                            #to
    [27, 16, 0, 0],                                      #cell
    [26, 0, 14, 27, 23],                                 #block
    [5, 5, 2, 24],                                       #??is
    [20, 22, 19, 5, 18, 10, 19, 19, 13, 20],             #??????????
    [27, 25, 0, 0],                                      #call
    [11, 7, 16],                                         #?he
    [0, 16, 8, 17, 3, 14, 24, 17, 1, 7, 14, 12, 16],     #leftmost?ho?e
    [2, 12, 17, 7, 16],                                  #i?the
    [28, 2, 24, 2, 17, 14, 4],                           #visitor
    [4, 14, 14, 3],                                      #room
]

In [3129]:
from english_words import english_words_lower_alpha_set

In [3130]:
ALPH = list(set(list("eariotnslcudpmhgbfywkvxzjqear")))

In [3131]:
from collections import defaultdict, Counter

In [3132]:
posschars = defaultdict(lambda: ALPH)

In [3133]:
# posschars[2] = ['i'] # ['i', 'a']
posschars[2] = ['i', 'a']
posschars[4] = ['r']
posschars[14] = ['o']
posschars[3] = ['m']

posschars[7] = ['h']
posschars[16] = ['e']
posschars[23] = ['k']

posschars[28] = ['v']
posschars[24] = ['s']
posschars[17] = ['t']

# posschars[11] = ['y']
# posschars[15] = ['u']

# # [8, 14, 4]
# posschars[8] = ['f']

# #keeping 
# # [23, 16, 16, 1, 2, 12, 6]
# posschars[1] = ["p"]
# posschars[12] = ["n"]
# posschars[6] = ["g"]

# #safe
# posschars[25] = ["f"]

# #tosolve
# posschars[0] = ["l"]

# posschars[27] = ["c"]
# posschars[9] = ["d"]

# posschars[26] = ["b"]
# posschars[21] = ["w"]
# posschars[25] = ['a']
# posschars[11] = ['y']

# posschars[5] = ['1']

#Eleventh
# posschars[20] = ['e']
# posschars[22] = ['l']
# posschars[19] = ['n']
# posschars[18] = ['h']

In [3134]:
def list_duplicates(seq):
    tally = defaultdict(list)
    for i,item in enumerate(seq):
        tally[item].append(i)
    return ((key,locs) for key,locs in tally.items() 
                            if len(locs)>1)

In [3135]:
def dynPred(word, posscharmap = None, mode="perfect"):
    global ALPH
    tot = 0.
    max_char_idx = -1

    # lamstr = "lambda x: " # add this yourself
    lamstr = ""
    if mode == "perfect": lamstr += f"(len(x)=={len(word)})"
    elif mode == "appx": lamstr += f"float(len(x)=={len(word)})"
    tot += 1.
    if len(word) != len(set(word)):
        #we have duplicates, create constraints
        for k, dupindxs in list_duplicates(word):
            first = None
            for dupindx in dupindxs:
                if first == None:
                    first = dupindx
                    continue
                if mode == "perfect": lamstr += f" and (x[{first}]==x[{dupindx}])"
                elif mode == "appx": lamstr += f" + float(x[{first}]==x[{dupindx}])"
                max_char_idx = max(first, max_char_idx)
                max_char_idx = max(dupindx, max_char_idx)
                tot += 1.
    
    for i in range(len(word)):
        for j in range(len(word)):
            if word[i] != word[j]:
                max_char_idx = max(i, max_char_idx)
                max_char_idx = max(j, max_char_idx)
                if mode == "perfect": lamstr += f" and (x[{i}]!=x[{j}])"
                elif mode == "appx": lamstr += f" + float(x[{i}]!=x[{j}])"

    for i, charid in enumerate(word):
        # print(posscharmap[charid])
        # print(charid)
        # print(posscharmap[charid])
        if len(posscharmap[charid]) < len(ALPH):
            if len(posscharmap[charid]) == 1:
                if mode == "perfect": lamstr += f" and (x[{i}]=='{posscharmap[charid][0]}')"
                elif mode == "appx": lamstr += f" + float(x[{i}]=='{posscharmap[charid][0]}')"
                max_char_idx = max(i, max_char_idx)
                tot += 1.
            else:
                strlist = '['+', '.join(map(lambda x: '\''+x+'\'', posscharmap[charid])) + ']'
                if mode == "perfect": lamstr += f" and (x[{i}] in {strlist})"
                elif mode == "appx": lamstr += f" + float(x[{i}] in {strlist})"
                max_char_idx = max(i, max_char_idx)
                tot += 1.

    if mode == "appx": return f"({lamstr})/{tot}", max_char_idx
    
    return lamstr, max_char_idx

In [3136]:
def fullgen(word, posschars, matchp = 1., verb=False):
    if matchp >= 1.:
        lambbody, maxacc = dynPred(word, posschars, 'perfect')
        if verb: print(f"querying 'lambda x: {lambbody}'")
        return f"lambda x: {lambbody}"
    else:
        lambbody, maxacc = dynPred(word, posschars, 'appx')
        if verb: print(f"querying 'lambda x: (len(x) > {maxacc}) and ({lambbody} > {matchp})'")
        return f"lambda x: (len(x) > {maxacc}) and ({lambbody} > {matchp})"

In [3137]:
def chars(words):
    longest = max(map(lambda x: len(x), words))
    charhist = []
    for i in range(longest):
        charhist.append(Counter())

    for word in words:
        for j in range(longest):
            if j >= len(word):
                charhist[j][None] += 1
            else:
                charhist[j][word[j]] += 1
    return charhist

In [3138]:
def updateposschars(word, charshist):
    updatedchars = {}
    for i in range(len(word)):
        updatedchars[word[i]] = list(set(charshist[i].keys()))
    return updatedchars

In [3139]:
# posschars

In [3140]:
# lambbody, maxacc = dynPred([20, 22, 19, 5, 18, 10, 19, 13, 20], posschars, 'appx')
# lambbody, maxacc = dynPred([17, 14, 24, 14, 0, 28, 16], posschars, 'appx')
# lambbody, maxacc = dynPred([23, 16, 16, 1, 2, 12, 6], posschars, 'appx')
# lambs = f"lambda x: (len(x) > {maxacc}) and ({lambbody} > .5)"

# lambbody, maxacc = dynPred([17, 14, 24, 14, 0, 28, 16], posschars, 'perfect')
# lambs = f"lambda x: {lambbody}"

In [3141]:
lambs = fullgen([4, 14, 14, 3], posschars, 1.)

In [3142]:
lambs

"lambda x: (len(x)==4) and (x[1]==x[2]) and (x[0]!=x[1]) and (x[0]!=x[2]) and (x[0]!=x[3]) and (x[1]!=x[0]) and (x[1]!=x[3]) and (x[2]!=x[0]) and (x[2]!=x[3]) and (x[3]!=x[0]) and (x[3]!=x[1]) and (x[3]!=x[2]) and (x[0]=='r') and (x[1]=='o') and (x[2]=='o') and (x[3]=='m')"

In [3143]:
cuslamb = eval(lambs)

In [3144]:
# results = list(filter(cuslamb, english_words_lower_alpha_set))
# results

In [3145]:
# tdict = updateposschars([4, 14, 14, 3], chars(results))
# for k, v in tdict.items():
#     posschars[k] = list(set(posschars[k]).intersection(set(v)))

In [3146]:
# WORDID = 12
# lambs = fullgen(wordList[WORDID], posschars, 1.)
# cuslamb = eval(lambs)
# results = list(filter(cuslamb, english_words_lower_alpha_set))
# posschars.update(updateposschars(wordList[WORDID], chars(results)))

In [3147]:
# posschars

In [3148]:
def tryguess(arr, guessmap):
    defaultedguesses = defaultdict(lambda: "?")
    for k, v in guessmap.items():
        if len(v) == 1:
            defaultedguesses[k] = v[0]
    return list(map(lambda x: defaultedguesses.__getitem__(x), arr))

In [3149]:
posschars[-1] = [" "]

In [3150]:
''.join(tryguess(sum([word + [-1] for word in wordList], start=[]), posschars))

'? th?k ?o? ?or kee???? me s??e ?h??e?o? ?o??? ?h?t ?o? ?ee?e? toso?ve th?s the ?o?e to ?e?? ??o?k ???s ?????????? ???? ?he ?e?tmost?ho?e ??the v?s?tor room '

In [3151]:
# for word in sorted(wordList, key=lambda x: len(x)):
#     lambs = fullgen(word, posschars, .2)
#     cuslamb = eval(lambs)
#     results = list(filter(cuslamb, english_words_lower_alpha_set))
#     posschars.update(updateposschars(word, chars(results)))
#     print(''.join(tryguess(sum([word + [-1] for word in wordList], start=[]), posschars)))

In [3152]:
import random

In [3153]:
reduced_eng = list(filter(lambda x: x not in ['aaas', 'algae', 'allele', 'chromatograph', 'tor', 'mn', 'mt', 'mo', 'md', 'mu', 'mi', 'ma', 'nor', 'leeward', 'assuage', 'ms', 'mr', 'joe', 'lee', 'des', 'poe', 'ho', 'jo', 'io', 'co', 'lo', 'po', 'fount'], english_words_lower_alpha_set))

In [3154]:
def runon(word):
    global posschars, reduced_eng, posschars, wordList
    lambs = fullgen(word, posschars, 1)
    cuslamb = eval(lambs)
    results = list(filter(cuslamb, reduced_eng))
    tdict = updateposschars(word, chars(results))
    for k, v in tdict.items():
        posschars[k] = list(set(posschars[k]).intersection(set(v)))
    #posschars.update(updateposschars(word, chars(results)))
    print(''.join(tryguess(sum([word + [-1] for word in wordList], start=[]), posschars)))

In [3155]:
# for i in range(5):
#     #sorted(wordList, key=lambda x: len(x)):
#     #sorted(wordList, key=lambda x: random.random())
#     for word in sorted(wordList, key=lambda x: random.random()):
#         try:
#             runon(word)
#         except:
#             pass
# print("Done")

In [3156]:
#reduced_eng = english_words_lower_alpha_set

for word in sorted(wordList, key=lambda x: random.random()):
    runon(word)
    # lambs = fullgen(word, posschars, 1)
    # cuslamb = eval(lambs)
    # results = list(filter(cuslamb, reduced_eng))
    # tdict = updateposschars(word, chars(results))
    # for k, v in tdict.items():
    #     posschars[k] = list(set(posschars[k]).intersection(set(v)))
    # #posschars.update(updateposschars(word, chars(results)))
    # print(''.join(tryguess(sum([word + [-1] for word in wordList], start=[]), posschars)))

? th?k ?o? for kee???? me s?fe ?h??e?o? fo??? ?h?t ?o? ?ee?e? toso?ve th?s the ?o?e to ?e?? ??o?k ???s ?????????? ???? ?he ?eftmost?ho?e ??the v?s?tor room 
? th?k ?o? for kee???? me s?fe ?h??e?o? fo??? ?h?t ?o? ?ee?e? toso?ve th?s the ?o?e to ?e?? ??o?k ???s ?????????? ???? ?he ?eftmost?ho?e ??the v?s?tor room 
? th?k ?o? for kee???? me s?fe ?h??e?o? fo??? ?h?t ?o? ?ee?e? toso?ve th?s the ?o?e to ?e?? ??o?k ???s ?????????? ???? ?he ?eftmost?ho?e ??the v?s?tor room 
? th?k ?o? for kee???? me s?fe ?h??e?o? fo??? ?h?t ?o? ?ee?e? toso?ve th?s the ?o?e to ?e?? ??o?k ???s ?????????? ???? ?he ?eftmost?ho?e ??the v?s?tor room 
? th?k ?o? for kee???? me s?fe ?h??e?o? fo??? ?h?t ?o? ?ee?e? toso?ve th?s the ?o?e to ?e?? ??o?k ???s ?????????? ???? ?he ?eftmost?ho?e ??the v?s?tor room 
? th?k ?o? for kee???? me s?fe ?h??e?o? fo??? ?h?t ?o? ?ee?e? toso?ve th?s the ?o?e to ?e?? ??o?k ???s ?????????? ???? ?he ?eftmost?ho?e ??the v?s?tor room 


ValueError: max() arg is an empty sequence

In [2010]:
# posschars

In [2011]:
lambs = fullgen(wordList[-1], posschars, 1)
cuslamb = eval(lambs)
results = list(filter(cuslamb, reduced_eng))
print(f"{len(results)} results")
results

98 results


['seen',
 'jeep',
 'beep',
 'good',
 'haas',
 'moor',
 'ally',
 'reed',
 'boom',
 'deep',
 'feed',
 'meet',
 'seep',
 'feel',
 'pooh',
 'doom',
 'abbe',
 'leer',
 'seek',
 'teem',
 'deem',
 'reek',
 'beef',
 'nook',
 'boor',
 'week',
 'reef',
 'zoom',
 'veer',
 'keel',
 'heed',
 'rood',
 'mood',
 'loop',
 'leek',
 'loot',
 'reel',
 'beer',
 'ammo',
 'weed',
 'weep',
 'wood',
 'moot',
 'iffy',
 'book',
 'roof',
 'peel',
 'root',
 'cool',
 'fool',
 'foot',
 'boot',
 'wool',
 'feet',
 'poor',
 'loon',
 'cook',
 'took',
 'coot',
 'look',
 'beet',
 'soot',
 'moon',
 'pool',
 'door',
 'soon',
 'seem',
 'hood',
 'hook',
 'seed',
 'tool',
 'room',
 'ella',
 'eddy',
 'coon',
 'anne',
 'hoof',
 'keen',
 'loom',
 'peer',
 'emma',
 'need',
 'meek',
 'been',
 'haag',
 'heel',
 'boon',
 'food',
 'goof',
 'hoot',
 'rook',
 'keep',
 'peek',
 'hoop',
 'coop',
 'ussr',
 'deer',
 'teen']

In [2012]:
lambs = fullgen(wordList[-7], posschars, 1)
cuslamb = eval(lambs)
results = list(filter(cuslamb, reduced_eng))
print(f"{len(results)} results")
results

0 results


[]

In [2013]:
# runon(wordList[8])

In [2014]:
# posschars

In [2015]:
list(filter(lambda x: (len(x) == 4) and (x[0] == x[1]), reduced_eng))

['eeoc', 'ooze']

In [2016]:
cardinal = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve']

In [2017]:
list(filter(lambda x: (len(x) == 9), ordinals))

[]

In [2018]:
len([20, 22, 19, 5, 18, 10, 19, 19, 13, 20])

10

In [2019]:
ordinals = ['first', 'second', 'third', 'fourth', 'fifth', 'thirteenth']

In [2020]:
len('Twelfth')

7

In [2021]:
# list(filter(lambda x: (len(x) == 9), reduced_eng))

In [2022]:
list(filter(lambda x: (len(x) == 4) and (x[0] == x[1]), reduced_eng))

['eeoc', 'ooze']